<a href="https://colab.research.google.com/github/vantainguyen/Bayesian-optimization-/blob/main/1_Plain_DNN_for_magnetic_field_forward_problem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Plain-DNN for magnetic field 

## Navigating to the working directory containing the datasets and model

In [1]:
source_file = False

if source_file:
  from google.colab import drive
  drive.mount('/content/drive')
  source_file = '/content/drive/My Drive/Datasets_for_magfield'

else:
  source_file = '/home/opc/vantai/Datasets_for_magfield'



## Checking the current working GPU

In [ ]:
!nvidia-smi

Mon Oct 18 01:13:07 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    34W / 300W |   6745MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Loading the datasets and doing feature engineering 

In [ ]:
# Loading data

import pickle
import os

os.chdir(source_file)

with open('para_2m_e', 'rb') as file_para:
    para = pickle.load(file_para)
    
with open('output_field_2m_e', 'rb') as file_out:
    output_field = pickle.load(file_out)

In [ ]:
len(output_field)

2000000

In [ ]:
# Feature engineering

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

import random
import numpy as np
import pickle

random.seed(123)

para_np = np.array(para)
output_field_np = np.array(output_field)

x_train, x_test, y_train, y_test = train_test_split(para_np, output_field_np, test_size=.20, shuffle=True)

x_sScaler = StandardScaler()
x_sScaler.fit(x_train)
s_x_train_scaled = x_sScaler.transform(x_train)
s_x_test_scaled = x_sScaler.transform(x_test)

y_sScaler = StandardScaler()
y_sScaler.fit(y_train)
s_y_train_scaled = y_sScaler.transform(y_train)
s_y_test_scaled = y_sScaler.transform(y_test)


# Saving training, testing and scalers data


with open('re_s_x_train_scaled', 'wb') as filename:
  pickle.dump(s_x_train_scaled, filename)

with open('re_s_y_train_scaled', 'wb') as filename:
  pickle.dump(s_y_train_scaled, filename)

with open('re_s_x_test_scaled', 'wb') as filename:
  pickle.dump(s_x_test_scaled, filename)

with open('re_s_y_test_scaled', 'wb') as filename:
  pickle.dump(s_y_test_scaled, filename)

with open('re_x_sScaler', 'wb') as filename:
  pickle.dump(x_sScaler, filename)

with open('re_y_sScaler', 'wb') as filename:
  pickle.dump(y_sScaler, filename)

## Loading ready processed datasets

In [2]:
import pickle
import os

os.chdir(source_file)

with open('re_s_x_train_scaled', 'rb') as filename:
  s_x_train_scaled = pickle.load(filename)

with open('re_s_y_train_scaled', 'rb') as filename:
  s_y_train_scaled = pickle.load(filename)

with open('re_s_x_test_scaled', 'rb') as filename:
  s_x_test_scaled = pickle.load(filename)

with open('re_s_y_test_scaled', 'rb') as filename:
  s_y_test_scaled = pickle.load(filename)

## PlainDNN model development

In [3]:
# Plain DNN model


import tensorflow as tf
import numpy as np
from tensorflow.keras.layers import Add, Concatenate, Activation


# Define a customised loss function

def loss_func(y_true, y_pred): 
    
    # Root mean square loss
    squared_difference = tf.square(y_true - y_pred)
    mean_square_loss = tf.reduce_mean(squared_difference, axis=-1)
    root_mean_square_loss = tf.sqrt(mean_square_loss)
    
    return root_mean_square_loss 
 
def loss_func_with_weights(y_true, y_pred): 

    alpha = 3.0
    beta = 5.0
    sq = tf.square(y_true - y_pred)
    sq1 = tf.reshape(sq[:, 0], shape=(-1, 1)) 
    sq2 = tf.reshape(alpha*sq[:, 1], shape=(-1, 1))
    sq3 = tf.reshape(beta*sq[:, 2], shape=(-1, 1))
    sq4 = Concatenate(axis=-1)([sq1, sq2, sq3])
    mean_square_loss = tf.reduce_mean(sq4, axis=-1)
    root_mean_square_loss = tf.sqrt(mean_square_loss+1e-20)
    
    return root_mean_square_loss 

class PlainDNNModel(tf.keras.Model):
    def __init__(self):
        
        super(PlainDNNModel, self).__init__()
        
        self.dense1_1 = tf.keras.layers.Dense(796, activation=tf.nn.relu)
        self.dense1_2 = tf.keras.layers.Dense(796, activation=tf.nn.relu)
        self.dense1_3 = tf.keras.layers.Dense(796, activation=tf.nn.relu)


        self.dense2_1 = tf.keras.layers.Dense(796, activation=tf.nn.relu)
        self.dense2_2 = tf.keras.layers.Dense(796, activation=tf.nn.relu)
        self.dense2_3 = tf.keras.layers.Dense(796, activation=tf.nn.relu)


        self.dense3_1 = tf.keras.layers.Dense(796, activation=tf.nn.relu)
        self.dense3_2 = tf.keras.layers.Dense(796, activation=tf.nn.relu)
        self.dense3_3 = tf.keras.layers.Dense(796, activation=tf.nn.relu)
        self.dense3_4 = tf.keras.layers.Dense(796, activation=tf.nn.relu)
        self.dense3_o = tf.keras.layers.Dense(3)

        
    def call(self, inputs):
        
        x_1 = self.dense1_1(inputs)
        x_1 = self.dense1_2(x_1)
        x_1 = self.dense1_3(x_1)

        
        x_2 = self.dense2_1(x_1)
        x_2 = self.dense2_2(x_2)
        x_2 = self.dense2_3(x_2)

        x_3 = self.dense3_1(x_2)
        x_3 = self.dense3_2(x_3)
        x_3 = self.dense3_3(x_3)
        x_3 = self.dense3_4(x_3)
        x_3 = self.dense3_o(x_3)
        
        
        return x_3
    


PlainModel = PlainDNNModel()  


opt = tf.keras.optimizers.Adam(learning_rate=0.001, amsgrad=True, beta_1=.9, beta_2=.9999)

PlainModel.compile(optimizer=opt, loss=loss_func)

2021-10-22 07:04:09.000852: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2021-10-22 07:04:10.617917: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-10-22 07:04:10.620626: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2021-10-22 07:04:11.137577: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-10-22 07:04:11.138424: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla V100-SXM2-16GB computeCapability: 7.0
coreClock: 1.53GHz coreCount: 80 deviceMemorySize: 15.78GiB deviceMemoryBandwidth: 836.37GiB/s
2021-10-22 07:04:11.138472: I tensorflow/stream_executor/platform/defau

## Loading pre-trained model

In [ ]:
import tensorflow as tf
import os
import numpy as np
from tensorflow.keras.layers import Add, Concatenate, Activation


# Define a customised loss function

def loss_func(y_true, y_pred): 
    
    # Root mean square loss
    squared_difference = tf.square(y_true - y_pred)
    mean_square_loss = tf.reduce_mean(squared_difference, axis=-1)
    root_mean_square_loss = tf.sqrt(mean_square_loss + 1e-20)
    
    return root_mean_square_loss 
 
def loss_func_with_weights(y_true, y_pred): 

    alpha = 3.0
    beta = 5.0
    sq = tf.square(y_true - y_pred)
    sq1 = tf.reshape(sq[:, 0], shape=(-1, 1)) 
    sq2 = tf.reshape(alpha*sq[:, 1], shape=(-1, 1))
    sq3 = tf.reshape(beta*sq[:, 2], shape=(-1, 1))
    sq4 = Concatenate(axis=-1)([sq1, sq2, sq3])
    mean_square_loss = tf.reduce_mean(sq4, axis=-1)
    root_mean_square_loss = tf.sqrt(mean_square_loss+1e-20)
    
    return root_mean_square_loss 

source_file_model = os.path.join(source_file, 're_HDNNmodel')
if os.getcwd != source_file_model:
  os.chdir(source_file_model)

PlainModel = PlainDNNModel()  
PlainModel.load_weights('PlainDNN_3001_noco')


opt = tf.keras.optimizers.Adam(learning_rate=0.001, amsgrad=True, beta_1=.9, beta_2=.9999 )
#Hmodel = tf.keras.models.load_model('HDNN_300', compile=False)

PlainModel.compile(optimizer=opt, loss=loss_func) # the loss_func is good enough we don't need the loss_func_with_weights


In [ ]:
Hmodel = tf.keras.models.load_model('HDNN_300', compile=False)


TypeError: ignored

In [ ]:
os.getcwd()

'/home/opc/vantai/Datasets_for_magfield/re_HDNNmodel'

In [ ]:
decay_steps = 40
step = [10, 100, 200, 300]
decay_rate = 0.5

for step in step:
  print(decay_rate**(step/decay_steps))

0.8408964152537145
0.1767766952966369
0.03125
0.005524271728019903


## Model training

In [4]:
import pickle
import time
import os
from datetime import datetime
%load_ext tensorboard

now = datetime.now()

source_file_model = os.path.join(source_file, 're_HDNNmodel')
if os.getcwd != source_file_model:
  os.chdir(source_file_model)



filepath = os.path.join(os.getcwd(), 'PlainDNN_400_796')

modelcheckpoint = tf.keras.callbacks.ModelCheckpoint(filepath=filepath, monitor='val_loss', 
                                                     mode='min', verbose=1, save_best_only=True, save_weights_only=True)

earlystopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', verbose=1, mode='min', patience=20, min_delta=0.00005)

new_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', mode='min', verbose=1, patience=10, 
                                              factor=1/10, min_lr=1e-5)

path = "logs/fit/" + now.strftime("%Y/%m/%/d, %H/%M")

#tensorboard = tf.keras.callbacks.TensorBoard(log_dir=path, histogram_freq=1)

st = time.time()

history = PlainModel.fit(s_x_train_scaled, s_y_train_scaled, validation_split=.20, 
                     batch_size=512, epochs=400, callbacks=[earlystopping, modelcheckpoint, new_lr])

used_time = time.time() - st

with open('p796_history_400', 'wb') as filename:
  pickle.dump(history.history, filename)

with open('p796_used_time_400', 'wb') as filename:
  pickle.dump(used_time, filename)

#oss: 0.1562 - val_loss: 0.1343

2021-10-22 07:04:42.512699: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2021-10-22 07:04:42.513375: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 1999995000 Hz


Epoch 1/400


2021-10-22 07:04:43.332088: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10


2500/2500 [==============================] - 12s 4ms/step - loss: 0.1765 - val_loss: 0.1000

Epoch 00001: val_loss improved from inf to 0.10003, saving model to /home/opc/vantai/Datasets_for_magfield/re_HDNNmodel/PlainDNN_400_796
Epoch 2/400
2500/2500 [==============================] - 11s 4ms/step - loss: 0.0956 - val_loss: 0.0774

Epoch 00002: val_loss improved from 0.10003 to 0.07741, saving model to /home/opc/vantai/Datasets_for_magfield/re_HDNNmodel/PlainDNN_400_796
Epoch 3/400
2500/2500 [==============================] - 11s 4ms/step - loss: 0.0853 - val_loss: 0.0885

Epoch 00003: val_loss did not improve from 0.07741
Epoch 4/400
2500/2500 [==============================] - 10s 4ms/step - loss: 0.0716 - val_loss: 0.0638

Epoch 00004: val_loss improved from 0.07741 to 0.06380, saving model to /home/opc/vantai/Datasets_for_magfield/re_HDNNmodel/PlainDNN_400_796
Epoch 5/400
2500/2500 [==============================] - 10s 4ms/step - loss: 0.0733 - val_loss: 0.0646

Epoch 00005: val_

In [ ]:
PlainModel.summary()

Model: "plain_dnn_model_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_110 (Dense)            multiple                  7960      
_________________________________________________________________
dense_111 (Dense)            multiple                  634412    
_________________________________________________________________
dense_112 (Dense)            multiple                  634412    
_________________________________________________________________
dense_113 (Dense)            multiple                  634412    
_________________________________________________________________
dense_114 (Dense)            multiple                  634412    
_________________________________________________________________
dense_115 (Dense)            multiple                  634412    
_________________________________________________________________
dense_116 (Dense)            multiple           

In [ ]:
%tensorboard --logdir logs/fit

Reusing TensorBoard on port 6006 (pid 27511), started 0:01:47 ago. (Use '!kill 27511' to kill it.)

## Model testing

In [5]:
# Predicting the results
s_y_predicted_scaled = PlainModel.predict(s_x_test_scaled)

In [8]:
# Testing the model
from sklearn.metrics import r2_score

r2_score(s_y_test_scaled[:, 2], s_y_predicted_scaled[:, 2])

0.9743515739083507